# Down sampling とbaggingを試してみる。
reference:https://upura.hatenablog.com/entry/2019/01/12/193000

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
folder_path = '/tmp/working/IEEE_Fraud_Detection/data/input/'
train_identity = pd.read_csv(f'{folder_path}train_identity.csv')
train_transaction = pd.read_csv(f'{folder_path}train_transaction.csv')
test_identity = pd.read_csv(f'{folder_path}test_identity.csv')
test_transaction = pd.read_csv(f'{folder_path}test_transaction.csv')
sub = pd.read_csv(f'{folder_path}sample_submission.csv')
# let's combine the data and work with the whole dataset
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

del train_identity, train_transaction, test_identity, test_transaction

one_value_cols = [col for col in train.columns if train[col].nunique() <= 1]
one_value_cols_test = [col for col in test.columns if test[col].nunique() <= 1]
one_value_cols == one_value_cols_test

False

In [3]:
## この特徴量は無理
train['TransactionAmt_to_mean_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_mean_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_std_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('std')
train['TransactionAmt_to_std_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('std')

test['TransactionAmt_to_mean_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_mean_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_std_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('std')
test['TransactionAmt_to_std_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('std')


#この特徴量はいける！！
train['id_02_to_mean_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('mean')
train['id_02_to_mean_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('mean')
train['id_02_to_std_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('std')
train['id_02_to_std_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('std')

test['id_02_to_mean_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('mean')
test['id_02_to_mean_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('mean')
test['id_02_to_std_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('std')
test['id_02_to_std_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('std')

## この特徴量は無理
train['D15_to_mean_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('mean')
train['D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
train['D15_to_std_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('std')
train['D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')


test['D15_to_mean_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('mean')
test['D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
test['D15_to_std_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('std')
test['D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')

# この特徴量は無理 
train['D15_to_mean_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('mean')
train['D15_to_mean_addr2'] = train['D15'] / train.groupby(['addr2'])['D15'].transform('mean')
train['D15_to_std_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('std')
train['D15_to_std_addr2'] = train['D15'] / train.groupby(['addr2'])['D15'].transform('std')

test['D15_to_mean_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('mean')
test['D15_to_mean_addr2'] = test['D15'] / test.groupby(['addr2'])['D15'].transform('mean')
test['D15_to_std_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('std')
test['D15_to_std_addr2'] = test['D15'] / test.groupby(['addr2'])['D15'].transform('std')

#この特徴量はいける！！
train[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = train['P_emaildomain'].str.split('.', expand=True)
train[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = train['R_emaildomain'].str.split('.', expand=True)
test[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = test['P_emaildomain'].str.split('.', expand=True)
test[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = test['R_emaildomain'].str.split('.', expand=True)

many_null_cols = [col for col in train.columns if train[col].isnull().sum() / train.shape[0] > 0.9]
many_null_cols_test = [col for col in test.columns if test[col].isnull().sum() / test.shape[0] > 0.9]

big_top_value_cols = [col for col in train.columns if train[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
big_top_value_cols_test = [col for col in test.columns if test[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]




cols_to_drop = list(set(many_null_cols + many_null_cols_test + big_top_value_cols + big_top_value_cols_test + one_value_cols+ one_value_cols_test))
cols_to_drop.remove('isFraud')
len(cols_to_drop)

train = train.drop(cols_to_drop, axis=1)
test = test.drop(cols_to_drop, axis=1)

cat_cols = ['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
            'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'ProductCD', 'card4', 'card6', 'M4','P_emaildomain',
            'R_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9',
            'P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3', 'R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']

for col in cat_cols:
    if col in train.columns:
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))

In [8]:
train_fr = train.loc[train['isFraud'] == 1]
train_nofr = train.loc[train['isFraud'] == 0]

In [20]:
train[train['isFraud']==1].shape[0]/(train[train['isFraud']==0].shape[0]+train[train['isFraud']==1].shape[0])

0.03499000914417313

In [22]:
train_nofr_dwsp = train_nofr.sample(n = train_fr.shape[0]*9, replace=True, random_state=0 )

In [23]:
train_fr.shape[0]/(train_nofr_dwsp.shape[0] + train_fr.shape[0])

0.1

In [26]:
train_dwsp = pd.concat([train_fr, train_nofr_dwsp]).sample(frac=1, random_state=0)
train_dwsp.head(20)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,...,id_13,id_14,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,TransactionAmt_to_mean_card1,TransactionAmt_to_mean_card4,TransactionAmt_to_std_card1,TransactionAmt_to_std_card4,id_02_to_mean_card1,id_02_to_mean_card4,id_02_to_std_card1,id_02_to_std_card4,D15_to_mean_card1,D15_to_mean_card4,D15_to_std_card1,D15_to_std_card4,D15_to_mean_addr1,D15_to_mean_addr2,D15_to_std_addr1,D15_to_std_addr2,P_emaildomain_1,P_emaildomain_2,R_emaildomain_1,R_emaildomain_2
176610,3163610,1,3847676,335.000,4,2765,378,50,4,126,1,104,80,241.0,16,32,8.0,2.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,1.0,0.0,25.0,8.0,172.0,172.0,1.0,466.0,1.0,NaN,NaN,NaN,429.0,39.0,NaN,...,55,28,3,2,127,568,547,2,2,86,136,6,461,4,2,2,2,2,2,2740,1.065807,2.515736,0.722054,1.468007,NaN,NaN,NaN,NaN,3.108627,2.760370,2.672590,2.280365,2.625124,2.546567,2.267412,2.251963,15,2,25,7
21935,3008935,0,577925,80.950,4,13180,483,50,4,126,1,104,80,139.0,55,32,3.0,3.0,0.0,3.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,12.0,3.0,168.0,168.0,42.0,448.0,42.0,NaN,NaN,NaN,260.0,91.0,NaN,...,55,28,3,2,127,568,547,2,2,86,136,6,461,4,2,2,2,2,2,2740,0.359073,0.607907,0.204425,0.354732,NaN,NaN,NaN,NaN,3.060151,2.653746,2.164749,2.192282,2.523724,2.448202,2.179830,2.164977,44,2,25,7
100649,3087649,0,2039895,50.000,2,13180,483,50,4,126,1,394,80,NaN,16,24,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,588.583313,0.583333,0.0,NaN,NaN,...,39,4,0,0,65,438,424,0,0,26,155,2,316,3,1,0,1,0,0,1170,0.221787,0.375483,0.126266,0.219105,0.724552,0.488527,0.731074,0.531658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,2,19,2
241546,3228546,0,5705270,159.950,4,11410,327,50,2,17,2,412,80,NaN,16,32,2.0,4.0,0.0,2.0,4.0,0.0,0.0,5.0,0.0,3.0,0.0,9.0,2.0,275.0,256.0,28.0,73.0,28.0,NaN,NaN,NaN,73.0,NaN,NaN,...,55,28,3,2,127,568,547,2,2,86,136,6,461,4,2,2,2,2,2,2740,1.916949,1.208194,2.959460,0.629936,NaN,NaN,NaN,NaN,0.238762,0.473551,0.265760,0.367231,0.420761,0.398926,0.356774,0.352775,15,2,25,7
177540,3164540,0,3871307,59.000,4,9994,412,50,4,126,2,91,80,NaN,32,32,1.0,2.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,2.0,0.0,4.0,1.0,43.0,43.0,4.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,55,28,3,2,127,568,547,2,2,86,136,6,461,4,2,2,2,2,2,2740,0.479566,0.443070,0.316431,0.258544,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25,7,25,7
230262,3217262,0,5449395,930.950,4,5479,70,50,2,2,1,215,80,NaN,32,32,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,55,28,3,2,127,568,547,2,2,86,136,6,461,4,2,2,2,2,2,2740,4.436024,7.031996,2.992514,3.666389,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25,7,25,7
185365,3172365,0,4127474,97.000,4,14943,61,50,2,17,2,103,80,NaN,2,32,3.0,3.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,3.0,1.0,0.0,NaN,NaN,98.0,0.0,NaN,NaN,NaN,98.0,NaN,NaN,...,55,28,3,2,127,568,547,2,2,86,136,6,461,4,2,2,2,2,2,2740,0.863304,0.732696,0.533580,0.382018,NaN,NaN,NaN,NaN,0.552914,0.635726,0.491996,0.492995,0.547658,0.535544,0.464356,0.473589,2,2,25,7
138557,3125557,0,2830168,20.950,4,6965,198,50,4,126,2,84,80,19.0,2,32,124.0,93.0,0.0,66.0,81.0,0.0,0.0,60.0,0.0,73.0,0.0,365.0,100.0,4.0,4.0,4.0,378.0,0.0,NaN,NaN,NaN,95.0,378.0,NaN,...,55,28,3,2,127,568,547,2,2,86,136,6,461,4,2,2,2,2,2,2740,0.154048,0.157327,0.074721,0.091805,NaN,NaN,NaN,NaN,2.249803,2.239098,1.964949,1.849738,1.887491,2.065670,1.774836,1.826699,2,2,25,7
415824,3402824,0,10511156,57.950,4,14796,390,50,4,126,2,164,80,NaN,16,32,154.0,161.0,0.0,255.0,105.0,0.0,0.0,119.0,0.0,110.0,0.0,752.0,134.0,179.0,56.0,1.0,0.0,NaN,NaN,NaN,NaN,0.0,438.0,NaN,...,55,28,3,2,127,568,547,2,2,86,136,6,461,4,2,2,2,2,2,2740,0.467893,0.435185,0.312889,0.253943,NaN,NaN,NaN,NaN,2.515054,3.050623,

In [4]:
X = train.sort_values('TransactionDT').drop(['isFraud', 'TransactionDT', 'TransactionID'], axis=1)
y = train.sort_values('TransactionDT')['isFraud']
# X_test = test.sort_values('TransactionDT').drop(['TransactionDT', 'TransactionID'], axis=1)
# del train
test = test[["TransactionDT", 'TransactionID']]

In [5]:
X.describe().T[X.describe().T['max'] == np.inf]

,count,mean,std,min,25%,50%,75%,max
TransactionAmt_to_std_card1,587096.0,inf,NaN,0.002421,0.271281,0.512906,1.059265,inf
D15_to_std_card1,495034.0,inf,NaN,-0.727011,0.000000,0.324561,1.774351,inf
D15_to_std_addr1,439441.0,inf,NaN,-0.287247,0.000000,0.408718,1.707858,inf


In [6]:
# describe(set(cat_cols)-set(['id_18', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27',
#                           'P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3', 'R_emaildomain_1',
#                           'R_emaildomain_2', 'R_emaildomain_3']))].dtypes

In [7]:
import gc 
gc.collect()

80

In [8]:
for col in X.columns.values:
#     if X[col].dtypes == np.int:
#         X[col].fillna(1, inplace=True)
    if X[col].dtypes == np.float:
        X[col].fillna(-999, inplace=True)

In [9]:
# X.iloc[0, 0] = np.nan

In [10]:
np.isnan(X.values).any()

False

In [11]:
X.head()

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,...,id_13,id_14,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,TransactionAmt_to_mean_card1,TransactionAmt_to_mean_card4,TransactionAmt_to_std_card1,TransactionAmt_to_std_card4,id_02_to_mean_card1,id_02_to_mean_card4,id_02_to_std_card1,id_02_to_std_card4,D15_to_mean_card1,D15_to_mean_card4,D15_to_std_card1,D15_to_std_card4,D15_to_mean_addr1,D15_to_mean_addr2,D15_to_std_addr1,D15_to_std_addr2,P_emaildomain_1,P_emaildomain_2,R_emaildomain_1,R_emaildomain_2
0,68.5,4,4248,501,50,1,42,1,215,80,19.0,32,32,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,-999.0,13.0,-999.0,-999.0,-999.0,-999.0,-999.0,13.0,13.0,-999.0,-999.0,-999.0,0.0,...,55,28,3,2,127,568,547,2,2,86,136,6,461,4,2,2,2,2,2,2740,0.194640,0.257761,0.184566,0.170233,-999.000000,-999.000000,-999.000000,-999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25,7,25,7
1,29.0,4,9979,304,50,2,2,1,225,80,-999.0,16,32,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,0.0,...,55,28,3,2,127,568,547,2,2,86,136,6,461,4,2,2,2,2,2,2740,0.123777,0.219054,0.062995,0.114212,-999.000000,-999.000000,-999.000000,-999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15,2,25,7
2,59.0,4,11850,390,50,4,66,2,230,80,287.0,36,32,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,0.0,315.0,-999.0,-999.0,-999.0,315.0,...,55,28,3,2,127,568,547,2,2,86,136,6,461,4,2,2,2,2,2,2740,0.608150,0.443070,0.589241,0.258544,-999.000000,-999.000000,-999.000000,-999.000000,2.518583,1.865915,1.851016,1.541448,1.611525,1.721392,1.486472,1.522249,28,2,25,7
3,50.0,4,8796,467,50,2,17,2,376,80,-999.0,55,32,2.0,5.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,-999.0,-999.0,-999.0,84.0,-999.0,-999.0,-999.0,-999.0,111.0,...,55,28,3,2,127,568,547,2,2,86,136,6,461,4,2,2,2,2,2,2740,0.405133,0.377679,0.259447,0.196917,-999.000000,-999.000000,-999.000000,-999.000000,0.550272,0.720057,0.520531,0.558392,0.686169,0.606586,0.576638,0.536412,44,2,25,7
4,50.0,1,11687,414,50,2,2,1,320,80,-999.0,16,32,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,55,7,1,1,65,438,44,1,1,7,162,3,268,3,1,0,1,1,1,1565,0.515612,0.377679,0.882933,0.196917,0.764773,0.373295,1.753301,0.425884,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,15,2,25,7


In [12]:
X = X.replace([np.inf, -np.inf], np.nan)
X.describe().T[X.describe().T['max'] == np.inf]

,count,mean,std,min,25%,50%,75%,max


In [13]:
np.isnan(X.values).any()

True

In [14]:
X['TransactionAmt_to_std_card1'].replace(to_replace=np.nan, value=X['TransactionAmt_to_std_card1'].max(), inplace=True)
X['D15_to_std_card1'].replace(to_replace=np.nan, value=X['D15_to_std_card1'].max(), inplace=True)
X['D15_to_std_addr1'].replace(to_replace=np.nan, value=X['D15_to_std_addr1'].max(), inplace=True)

In [15]:
np.isnan(X.values).any()

False

In [16]:
from imblearn.under_sampling import RandomUnderSampler

sampler = RandomUnderSampler(random_state=42)
# downsampling
X_resampled, y_resampled = sampler.fit_resample(X, y)

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [17]:
X_resampled = pd.DataFrame(X_resampled, columns=X.columns)
X_resampled.head()

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,...,id_13,id_14,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,TransactionAmt_to_mean_card1,TransactionAmt_to_mean_card4,TransactionAmt_to_std_card1,TransactionAmt_to_std_card4,id_02_to_mean_card1,id_02_to_mean_card4,id_02_to_std_card1,id_02_to_std_card4,D15_to_mean_card1,D15_to_mean_card4,D15_to_std_card1,D15_to_std_card4,D15_to_mean_addr1,D15_to_mean_addr2,D15_to_std_addr1,D15_to_std_addr2,P_emaildomain_1,P_emaildomain_2,R_emaildomain_1,R_emaildomain_2
0,25.00,1.0,15525.0,164.0,50.0,4.0,126.0,2.0,225.0,80.0,-999.0,16.0,19.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.000,-999.000,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,39.0,4.0,1.0,1.0,65.0,426.0,438.0,1.0,1.0,73.0,130.0,3.0,80.0,2.0,1.0,0.0,0.0,1.0,1.0,2693.0,0.163495,0.187742,0.091335,0.109553,0.765792,0.509997,0.680392,0.555023,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,15.0,2.0,17.0,2.0
1,200.00,2.0,8741.0,11.0,50.0,2.0,119.0,1.0,227.0,80.0,-999.0,1.0,16.0,1.0,1.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0,1.0,24.0,24.0,-999.0,-999.0,-999.0,-999.0,747.625,0.625,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,42.0,4.0,0.0,0.0,65.0,426.0,77.0,0.0,0.0,49.0,49.0,2.0,216.0,3.0,1.0,0.0,1.0,0.0,0.0,2526.0,0.737431,1.510714,0.380622,0.787666,0.333787,0.211335,0.489783,0.241108,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,1.0,2.0,15.0,2.0
2,146.00,4.0,15525.0,164.0,50.0,4.0,126.0,2.0,172.0,80.0,-999.0,16.0,32.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.000,-999.000,0.0,-999.0,-999.0,-999.0,-999.0,0.0,...,55.0,28.0,3.0,2.0,127.0,568.0,547.0,2.0,2.0,86.0,136.0,6.0,461.0,4.0,2.0,2.0,2.0,2.0,2.0,2740.0,0.954812,1.096410,0.533397,0.639788,-999.000000,-999.000000,-999.000000,-999.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.0,2.0,25.0,7.0
3,57.95,4.0,1314.0,479.0,50.0,4.0,126.0,2.0,84.0,80.0,4.0,16.0,32.0,4.0,3.0,0.0,1.0,3.0,0.0,0.0,3.0,0.0,3.0,0.0,27.0,4.0,403.0,403.0,36.0,488.0,23.0,-999.0,-999.000,-999.000,488.0,434.0,-999.0,-999.0,-999.0,488.0,...,55.0,28.0,3.0,2.0,127.0,568.0,547.0,2.0,2.0,86.0,136.0,6.0,461.0,4.0,2.0,2.0,2.0,2.0,2.0,2740.0,0.465703,0.435185,0.249471,0.253943,-999.000000,-999.000000,-999.000000,-999.000000,2.688774,2.890688,2.452734,2.388021,2.436761,2.666791,2.291323,2.358279,15.0,2.0,25.0,7.0
4,49.00,4.0,2780.0,362.0,50.0,4.0,126.0,2.0,91.0,80.0,-999.0,55.0,32.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,12.0,1.0,39.0,39.0,0.0,38.0,0.0,-999.0,-999.000,-999.000,38.0,-999.0,-999.0,-999.0,-999.0,38.0,...,55.0,28.0,3.0,2.0,127.0,568.0,547.0,2.0,2.0,86.0,136.0,6.0,461.0,4.0,2.0,2.0,2.0,2.0,2.0,2740.0,0.470245,0.367973,0.268659,0.214723,-999.000000,-999.000000,-999.000000,-999.000000,0.317555,0.225095,0.207148,0.185952,0.195806,0.207660,0.183113,0.183636,44.0,2.0,25.0,7.0


In [18]:
y_resampled = pd.Series(y_resampled, name=y.name)
y_resampled.head()

0    0
1    0
2    0
3    0
4    0
Name: isFraud, dtype: int64

In [19]:
X_resampled.replace(to_replace=-999.0, value=np.nan, inplace=True)
X_resampled.head()

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,...,id_13,id_14,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,TransactionAmt_to_mean_card1,TransactionAmt_to_mean_card4,TransactionAmt_to_std_card1,TransactionAmt_to_std_card4,id_02_to_mean_card1,id_02_to_mean_card4,id_02_to_std_card1,id_02_to_std_card4,D15_to_mean_card1,D15_to_mean_card4,D15_to_std_card1,D15_to_std_card4,D15_to_mean_addr1,D15_to_mean_addr2,D15_to_std_addr1,D15_to_std_addr2,P_emaildomain_1,P_emaildomain_2,R_emaildomain_1,R_emaildomain_2
0,25.00,1.0,15525.0,164.0,50.0,4.0,126.0,2.0,225.0,80.0,NaN,16.0,19.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.0,4.0,1.0,1.0,65.0,426.0,438.0,1.0,1.0,73.0,130.0,3.0,80.0,2.0,1.0,0.0,0.0,1.0,1.0,2693.0,0.163495,0.187742,0.091335,0.109553,0.765792,0.509997,0.680392,0.555023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,2.0,17.0,2.0
1,200.00,2.0,8741.0,11.0,50.0,2.0,119.0,1.0,227.0,80.0,NaN,1.0,16.0,1.0,1.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0,1.0,24.0,24.0,NaN,NaN,NaN,NaN,747.625,0.625,NaN,NaN,NaN,NaN,NaN,NaN,...,42.0,4.0,0.0,0.0,65.0,426.0,77.0,0.0,0.0,49.0,49.0,2.0,216.0,3.0,1.0,0.0,1.0,0.0,0.0,2526.0,0.737431,1.510714,0.380622,0.787666,0.333787,0.211335,0.489783,0.241108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,15.0,2.0
2,146.00,4.0,15525.0,164.0,50.0,4.0,126.0,2.0,172.0,80.0,NaN,16.0,32.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,55.0,28.0,3.0,2.0,127.0,568.0,547.0,2.0,2.0,86.0,136.0,6.0,461.0,4.0,2.0,2.0,2.0,2.0,2.0,2740.0,0.954812,1.096410,0.533397,0.639788,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.0,2.0,25.0,7.0
3,57.95,4.0,1314.0,479.0,50.0,4.0,126.0,2.0,84.0,80.0,4.0,16.0,32.0,4.0,3.0,0.0,1.0,3.0,0.0,0.0,3.0,0.0,3.0,0.0,27.0,4.0,403.0,403.0,36.0,488.0,23.0,NaN,NaN,NaN,488.0,434.0,NaN,NaN,NaN,488.0,...,55.0,28.0,3.0,2.0,127.0,568.0,547.0,2.0,2.0,86.0,136.0,6.0,461.0,4.0,2.0,2.0,2.0,2.0,2.0,2740.0,0.465703,0.435185,0.249471,0.253943,NaN,NaN,NaN,NaN,2.688774,2.890688,2.452734,2.388021,2.436761,2.666791,2.291323,2.358279,15.0,2.0,25.0,7.0
4,49.00,4.0,2780.0,362.0,50.0,4.0,126.0,2.0,91.0,80.0,NaN,55.0,32.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,12.0,1.0,39.0,39.0,0.0,38.0,0.0,NaN,NaN,NaN,38.0,NaN,NaN,NaN,NaN,38.0,...,55.0,28.0,3.0,2.0,127.0,568.0,547.0,2.0,2.0,86.0,136.0,6.0,461.0,4.0,2.0,2.0,2.0,2.0,2.0,2740.0,0.470245,0.367973,0.268659,0.214723,NaN,NaN,NaN,NaN,0.317555,0.225095,0.207148,0.185952,0.195806,0.207660,0.183113,0.183636,44.0,2.0,25.0,7.0


In [20]:
len(X[y == 0]), len(X[y==1])

(569877, 20663)

In [21]:
 len(X[y==1])/len(X[y == 0])

0.03625870143908247

In [22]:
len(X_resampled[y_resampled == 0]), len(X_resampled[y_resampled==1])

(20663, 20663)

In [23]:
X.shape

(590540, 369)

In [24]:
y

0         0
1         0
2         0
3         0
4         0
         ..
590535    0
590536    0
590537    0
590538    0
590539    0
Name: isFraud, Length: 590540, dtype: int64

In [25]:
y_resampled

0        0
1        0
2        0
3        0
4        0
        ..
41321    1
41322    1
41323    1
41324    1
41325    1
Name: isFraud, Length: 41326, dtype: int64

In [26]:
len(X_resampled)

41326

In [27]:
X_resampled_s = X_resampled.sample(frac=1, random_state=0)
X_resampled_s.head()

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,P_emaildomain,R_emaildomain,C1,C2,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D8,D9,D10,D11,D12,D13,D14,D15,...,id_13,id_14,id_15,id_16,id_17,id_19,id_20,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,TransactionAmt_to_mean_card1,TransactionAmt_to_mean_card4,TransactionAmt_to_std_card1,TransactionAmt_to_std_card4,id_02_to_mean_card1,id_02_to_mean_card4,id_02_to_std_card1,id_02_to_std_card4,D15_to_mean_card1,D15_to_mean_card4,D15_to_std_card1,D15_to_std_card4,D15_to_mean_addr1,D15_to_mean_addr2,D15_to_std_addr1,D15_to_std_addr2,P_emaildomain_1,P_emaildomain_2,R_emaildomain_1,R_emaildomain_2
27144,22.912,0.0,4627.0,77.0,85.0,2.0,37.0,1.0,441.0,93.0,NaN,19.0,19.0,34.0,5.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,4.0,4.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.958333,0.958333,0.0,NaN,0.0,0.0,396.0,0.0,...,39.0,28.0,0.0,0.0,122.0,381.0,396.0,0.0,0.0,86.0,50.0,6.0,461.0,4.0,0.0,0.0,1.0,0.0,1.0,1818.0,0.576140,0.173067,0.587901,0.090235,0.869462,1.120412,1.241010,1.278254,0.00000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,17.0,2.0,17.0,2.0
26759,129.950,4.0,2916.0,322.0,50.0,4.0,126.0,2.0,287.0,80.0,9.0,32.0,32.0,57.0,13.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,7.0,0.0,25.0,2.0,302.0,302.0,5.0,NaN,NaN,NaN,NaN,NaN,160.0,297.0,NaN,NaN,NaN,303.0,...,55.0,28.0,3.0,2.0,127.0,568.0,547.0,2.0,2.0,86.0,136.0,6.0,461.0,4.0,2.0,2.0,2.0,2.0,2.0,2740.0,0.941945,0.975880,0.787433,0.569455,NaN,NaN,NaN,NaN,1.43497,1.794833,1.570088,1.482726,1.754491,1.655815,1.501363,1.464259,25.0,7.0,25.0,7.0
3656,25.022,0.0,6367.0,445.0,85.0,4.0,38.0,2.0,441.0,93.0,NaN,19.0,19.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,NaN,0.0,...,54.0,28.0,1.0,1.0,122.0,314.0,450.0,1.0,1.0,86.0,30.0,6.0,461.0,4.0,0.0,0.0,1.0,0.0,1.0,2091.0,0.634186,0.187907,0.691638,0.109649,2.094035,3.219694,3.281017,3.503950,0.00000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,17.0,2.0,17.0,2.0
322,49.000,4.0,13354.0,398.0,50.0,2.0,17.0,2.0,237.0,80.0,3.0,16.0,32.0,2.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,1.0,36.0,NaN,36.0,36.0,36.0,NaN,NaN,NaN,42.0,92.0,NaN,NaN,NaN,92.0,...,55.0,28.0,3.0,2.0,127.0,568.0,547.0,2.0,2.0,86.0,136.0,6.0,461.0,4.0,2.0,2.0,2.0,2.0,2.0,2740.0,0.470715,0.370125,0.285969,0.192978,NaN,NaN,NaN,NaN,0.31164,0.596804,0.375805,0.462811,0.489283,0.502756,0.435243,0.444593,15.0,2.0,25.0,7.0
21686,97.000,4.0,16577.0,462.0,50.0,2.0,2.0,1.0,412.0,80.0,NaN,32.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,55.0,28.0,3.0,2.0,127.0,568.0,547.0,2.0,2.0,86.0,136.0,6.0,461.0,4.0,2.0,2.0,2.0,2.0,2.0,2740.0,0.426448,0.732696,0.290119,0.382018,NaN,NaN,NaN,NaN,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.0,7.0,25.0,7.0


In [29]:
y_resampled_s = y_resampled.sample(frac=1, random_state=0)
y_resampled_s.head()

27144    1
26759    1
3656     0
322      0
21686    1
Name: isFraud, dtype: int64